In [1]:
import pandas as pd
import netCDF4 as nc
import numpy as np
import os
import geopandas as gpd

In [2]:
past_dirc='<CREST-VEC results for the Past time>'
future_dirc='<CREST-VEC results for the Future Time>'
network='<River network shpfile>'

In [3]:
length=pd.read_csv(network,usecols=['COMID', 'lengthkm','order_'],dtype={'COMID': int, 'lengthkm': float,'order_':int})
print(length)

        COMID   lengthkm  order_
0    67007971   3.600556       2
1    67007992   4.734361       4
2    67007993  15.025808       4
3    67007994   2.676237       4
4    67007995   3.451361       4
..        ...        ...     ...
749  67010249   9.630762       1
750  67010252   5.973005       1
751  67010254  12.218295       1
752  67010258   4.340974       1
753  67010260   4.222490       1

[754 rows x 3 columns]


In [3]:
# Open the NetCDF file
dataset = nc.Dataset(past_dirc)

In [4]:
past=dataset.variables['IRFroutedRunoff'][:]
comid=dataset.variables['reachID'][:]

In [5]:
# Open the NetCDF file
dataset = nc.Dataset(future_dirc)

future=dataset.variables['IRFroutedRunoff'][:]

In [6]:
past=past[330:,:]
future=future[330:,:]

In [8]:
print(np.shape(past))

(51463, 754)


In [11]:
_5yrevent=np.max(past,axis=0)
com_bi=np.sum(np.where(future > _5yrevent, 1,0),axis=0)/24
pdf=pd.DataFrame(np.transpose(np.vstack((comid,com_bi))),columns=['COMID','Days of floods']).to_csv('<output dir>')

In [12]:
past[past < 0.0003] = 0
past[past > 0.0003] = 1

future[future < 0.0003] = 0
future[future > 0.0003] = 1

past_freq=np.sum(past,axis=0)/51463*100
future_freq=np.sum(future,axis=0)/51463*100

In [30]:
pdf=pd.DataFrame(np.transpose(np.vstack((past_freq,future_freq))),columns=['Occurance: Past','Occurance: Future']).to_csv('<output dir>')

In [16]:
occur=np.transpose(np.vstack((past_freq,future_freq)))
occur=pd.DataFrame(np.hstack((comid,occur)),columns=['COMID','Occurance: Past','Occurance: Future'])
pdf=pd.merge(length,occur,on = 'COMID')
print(pdf)

        COMID   lengthkm  order_  Occurance: Past  Occurance: Future
0    67007971   3.600556       2        91.842683          60.221907
1    67007992   4.734361       4        94.423178          73.981307
2    67007993  15.025808       4        93.546820          73.897752
3    67007994   2.676237       4        93.348619          73.462488
4    67007995   3.451361       4        92.443114          73.260401
..        ...        ...     ...              ...                ...
749  67010249   9.630762       1        96.986184          62.790743
750  67010252   5.973005       1        76.726969          54.707265
751  67010254  12.218295       1        96.955094          63.717622
752  67010258   4.340974       1        88.348911          54.099061
753  67010260   4.222490       1        79.703865          53.494744

[754 rows x 5 columns]


In [17]:
chili_length=pd.read_csv(network_chili,usecols=['COMID', 'lengthkm'],dtype={'COMID': int, 'lengthkm': float})
calca_length=pd.read_csv(network_calca,usecols=['COMID', 'lengthkm'],dtype={'COMID': int, 'lengthkm': float})
chili_id=chili_length['COMID'].to_list()
calca_id=calca_length['COMID'].to_list()
pdf_ch=pdf[pdf.iloc[:, 0].isin(chili_id)]
pdf_ca=pdf[pdf.iloc[:, 0].isin(calca_id)]

In [22]:
pdf_out=pdf_ca.groupby('order_').mean()
print(pdf_out)

               COMID  lengthkm  Occurance: Past  Occurance: Future
order_                                                            
1       6.700950e+07  6.881571        70.845368          48.919902
2       6.700896e+07  6.733951        82.091914          64.446376
3       6.700892e+07  5.618401        89.752767          78.362642
4       6.700869e+07  6.111462        97.180346          82.236581
5       6.700898e+07  7.103616       100.000000          99.065939


In [3]:
network_chili='e:/PeruProject/crestVec/topo/Rio_Chili.csv'
network_calca='e:/PeruProject/crestVec/topo/Rio_Calca.csv'
chili_length=pd.read_csv(network_chili,usecols=['COMID', 'lengthkm'],dtype={'COMID': int, 'lengthkm': float})
calca_length=pd.read_csv(network_calca,usecols=['COMID', 'lengthkm'],dtype={'COMID': int, 'lengthkm': float})
dataset = nc.Dataset(past_dirc)
past=dataset.variables['IRFroutedRunoff'][:]
comid=dataset.variables['reachID'][:]
dataset = nc.Dataset(future_dirc)
future=dataset.variables['IRFroutedRunoff'][:]
dataset=[]

In [7]:
comid=comid.reshape(754,1)

In [8]:
pastT=pd.DataFrame(np.hstack((comid,np.transpose(past))))
futureT=pd.DataFrame(np.hstack((comid,np.transpose(future))))

In [9]:
colca_ids=[67008970,67009031,67008004]
chili_ids=[67009162,67009190,67009280]
chili_past=pastT[pastT.iloc[:, 0].isin(chili_ids)].to_numpy()
calca_past=pastT[pastT.iloc[:, 0].isin(colca_ids)].to_numpy()
chili_future=futureT[futureT.iloc[:, 0].isin(chili_ids)].to_numpy()
calca_future=futureT[futureT.iloc[:, 0].isin(colca_ids)].to_numpy()

In [18]:
print(calca_past)

[[6.70080040e+07 2.62996525e-01 1.06150830e+00 ... 1.93778553e+01
  1.94720192e+01 1.95598392e+01]
 [6.70089700e+07 4.11427736e-09 1.71228010e-08 ... 1.23596680e+02
  1.23538086e+02 1.23759964e+02]
 [6.70090310e+07 3.19888256e-03 1.23337675e-02 ... 7.29020920e+01
  7.32736053e+01 7.36218033e+01]]


In [10]:
chili=pd.DataFrame(np.transpose(np.vstack((chili_past,chili_future))),columns=['Past_Down','Past_Mid','Past_Up','Future_Down','Future_Mid','Future_Up']).to_csv('<output dir>')
colca=pd.DataFrame(np.transpose(np.vstack((calca_past,calca_future))),columns=['Past_Down','Past_Mid','Past_Up','Future_Down','Future_Mid','Future_Up']).to_csv('<output dir>')

In [6]:
chili_id=chili_length['COMID'].to_list()
calca_id=calca_length['COMID'].to_list()
# subsetting based on the basin
chili_past=pastT[pastT.iloc[:, 0].isin(chili_id)]
calca_past=pastT[pastT.iloc[:, 0].isin(calca_id)]
chili_future=futureT[futureT.iloc[:, 0].isin(chili_id)]
calca_future=futureT[futureT.iloc[:, 0].isin(calca_id)]

In [9]:
#subset to 2015-01-01 and convert to binary
past_chili = chili_past.iloc[:,330:].to_numpy()
past_calca = calca_past.iloc[:,330:].to_numpy()
future_chili = chili_future.iloc[:,330:].to_numpy()
future_calca = calca_future.iloc[:,330:].to_numpy()
# take median over time due to the voltility of the time seris, but take the mean over the space, which gives more hydrological meaning. To be discussed.
print(np.mean(np.median(past_chili,axis=1)),np.mean(np.median(past_calca,axis=1)),np.mean(np.median(future_chili,axis=1)),np.mean(np.median(future_calca,axis=1)))

1.8641071368914346 5.034417194479288 0.2581328701753635 0.8547973261282711


In [17]:
#subset to 2015-01-01 and convert to binary
past_chili = chili_past.iloc[:,330:].to_numpy()
past_calca = calca_past.iloc[:,330:].to_numpy()
future_chili = chili_future.iloc[:,330:].to_numpy()
future_calca = calca_future.iloc[:,330:].to_numpy()

#rio chili
chili_len = chili_length['lengthkm'].to_numpy()
chili_len = chili_len.reshape(336,1)
past_chili[past_chili < 0.0003] = 0
past_chili[past_chili >= 0.0003] = 1
future_chili[future_chili < 0.0003] = 0
future_chili[future_chili >= 0.0003] = 1

Rchili_len_past = np.sum(chili_len*past_chili, axis = 0)
Rchili_len_future = np.sum(chili_len*future_chili, axis = 0)

#rio calca
calca_len = calca_length['lengthkm'].to_numpy()
calca_len = calca_len.reshape(419,1)
past_calca[past_calca < 0.0003] = 0
past_calca[past_calca >= 0.0003] = 1
future_calca[future_calca < 0.0003] = 0
future_calca[future_calca >= 0.0003] = 1

Rcalca_len_past = np.sum(calca_len*past_calca, axis = 0)
Rcalca_len_future = np.sum(calca_len*future_calca, axis = 0)

pdf=pd.DataFrame(np.transpose(np.vstack((Rchili_len_past,Rchili_len_future,Rcalca_len_past,Rcalca_len_future))),columns = ['Rio Chili_Past','Rio Chili_Future','Rio Colca_Past','Rio Colca_Future']).to_csv('<output results>')